In [ ]:
import os
import pandas as pd
import nltk
import pickle as pick
import numpy as np
import random

from unidecode import unidecode
from string import punctuation
from nltk import ngrams

In [ ]:
# nltk.download('stopwords')
# nltk.download('rslp')

#nltk.download('stopwords')
# stopwords = nltk.corpus.stopwords.words('portuguese') #remove palavras desnecessarias

# nltk.download('rslp')
# stemmer = nltk.stem.RSLPStemmer()#deixa somente a raiz da palavra

In [ ]:
df = pd.read_csv(os.path.join('..', 'storage', 'base_sentiment.1.0.2.csv'))

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['classificacao'] = df.sentiment.replace(["negative", 'positive'], [0, 1])

In [ ]:
df['sentiment'].value_counts()

In [ ]:
all_words = ' '.join([text for text in df['text']])

In [ ]:
token_whitespace = nltk.tokenize.WhitespaceTokenizer()
token_frase = token_whitespace.tokenize(all_words) 
freq = nltk.FreqDist(token_frase)

In [ ]:
freq

In [ ]:
df_freq = pd.DataFrame({'palavra': list(freq.keys()), 'frequencia': list(freq.values())})

In [ ]:
df_freq

In [ ]:
df_freq.nlargest(columns='frequencia', n=10)

In [ ]:
palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')
frase_processada = list()

In [ ]:
for opiniao in df.text:
    nova_frase = list()
    palavra_texto = token_whitespace.tokenize(opiniao)

    for palavra in palavra_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    
    frase_processada.append(' '.join(nova_frase))

In [ ]:
df['tratamento_1'] = frase_processada
df.head()

In [ ]:
pontuacao = list()
frase_processada = list()

for ponto in punctuation:
    pontuacao.append(ponto)

palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')
pontuacao_stopwords = pontuacao + palavras_irrelevantes

for opiniao in df['tratamento_1']:
    nova_frase = list()
    palavra_texto = token_whitespace.tokenize(opiniao)

    for palavra in palavra_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)

    frase_processada.append(' '.join(nova_frase))

df['tratamento_2'] = frase_processada

In [ ]:
df.head()

In [ ]:
sem_acentos = [unidecode(texto) for texto in df['tratamento_2']]

In [ ]:
df['tratamento_3'] = sem_acentos

In [ ]:
stopwords_sem_acentos = [unidecode(texto) for texto in pontuacao_stopwords]
frase_processada = list()

for opiniao in df['tratamento_3']:
    nova_frase = list()
    palavra_texto = token_whitespace.tokenize(opiniao)

    for palavra in palavra_texto:
        if palavra not in stopwords_sem_acentos:
            nova_frase.append(palavra)

    frase_processada.append(' '.join(nova_frase))

df['tratamento_4'] = frase_processada

In [ ]:
df.head()

In [ ]:
frase_processada = list()
for opiniao in df['tratamento_4']:
    nova_frase = list()
    opiniao = opiniao.lower()
    palavra_texto = token_whitespace.tokenize(opiniao)

    for palavra in palavra_texto:
        if palavra not in stopwords_sem_acentos:
            nova_frase.append(palavra)

    frase_processada.append(' '.join(nova_frase))

df['tratamento_5'] = frase_processada

In [ ]:
df.head()

In [ ]:
stemmer = nltk.RSLPStemmer()

frase_processada = list()
for opiniao in df['tratamento_5']:
    nova_frase = list()
    palavra_texto = token_whitespace.tokenize(opiniao)

    for palavra in palavra_texto:
        nova_frase.append(stemmer.stem(palavra))

    frase_processada.append(' '.join(nova_frase))

df['tratamento_6'] = frase_processada

In [ ]:
df_tread = df[['classificacao', 'tratamento_6']]

In [ ]:
list_pos = df_tread.query('classificacao == 1').values
list_neg = df_tread.query('classificacao == 0').values

In [ ]:
def get_for_model(token_list):
    for tokens in token_list:
        yield dict([token, True] for token in tokens)

positive_tokens_for_model = get_for_model(list_pos)
negative_tokens_for_model = get_for_model(list_neg)

In [ ]:
positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

training_data = dataset[:int((len(dataset)) / 100 * 80)]
test_data = dataset[int((len(dataset)) / 100 * 20):]

In [ ]:
dataset

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_data)

print("Accuracy is:", nltk.classify.accuracy(classifier, test_data))

In [ ]:
custom_tweet = "oi! tudo bem? sim tudo ótimo"

custom_tokens = token_whitespace.tokenize(custom_tweet)

print(classifier.classify(dict([token, True] for token in custom_tokens)))